In [ ]:
import numpy as np
import pandas as pd
import torch
import torchvision
from torchvision import models, datasets, transforms
import torch.optim as optim
from tqdm import tqdm
import os
import cv2
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import scipy

import matplotlib.pylab as plt
from matplotlib import cm
%matplotlib inline

from tensorflow.keras.preprocessing import image

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from albumentations.pytorch import ToTensorV2
import albumentations as albu
import shutil

from torchvision import transforms

In [ ]:
disk_folder = '/kaggle/working/'
data_folder = '/kaggle/input/classification-of-handwritten-letters/'

In [ ]:
#Merger dataframes

def ohe_letter(label):
    result = np.zeros(len(letters))
    index = letters.index(label)
    result[index] = 1
    return result

main_df = pd.read_csv(data_folder + 'letters.csv')
main_df.drop('background', axis=1, inplace=True)

for i in range(len(main_df)):
    main_df['file'][i] = os.path.join('letters', main_df['file'][i])

for root in ['letters2', 'letters3']:
    tmp_df = pd.read_csv(data_folder + root + '.csv')
    tmp_df.drop('background', axis=1, inplace=True)
    for i in range(len(tmp_df)):
        tmp_df['file'][i] = os.path.join(root, tmp_df['file'][i])
    main_df = main_df.merge(tmp_df, how='outer')
    
main_df = shuffle(main_df)
main_df.head(), main_df.info()

In [ ]:
letters = ''
name_of_letters = ['filename']
for letter in main_df.letter.unique():
    letters += letter
    name_of_letters.append(letter)
    
labels = main_df.label

classnames = [i for i in letters]

inputs = main_df['letter'].apply(ohe_letter)
a = []
for i in range(len(inputs)):
    t = [main_df.file.values[i]]
    for q in inputs[i]:
        t.append(q)
    a.append(t)
a

main_df = shuffle(pd.DataFrame(a, columns = name_of_letters))

In [ ]:
for Class in classnames:
    for phase in ['train', 'valid']:
        os.makedirs(os.path.join(disk_folder, phase, Class))

In [ ]:
train_df, valid_df = train_test_split(main_df,
                               test_size=0.15,
                               random_state=42)

In [ ]:
for index, row in train_df.iterrows():
    for bukv in classnames:
        if row[bukv]:
            shutil.copy(os.path.join(data_folder, row['filename']), os.path.join(disk_folder, 'train', bukv))

for index, row in valid_df.iterrows():
    for bukv in classnames:
        if row[bukv]:
            shutil.copy(os.path.join(data_folder, row['filename']), os.path.join(disk_folder, 'valid', bukv))

In [ ]:
data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
    ]),
    'valid':
    transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
    ]),
}

image_datasets = {
    'train':
    datasets.ImageFolder(os.path.join(disk_folder, 'train'), data_transforms['train']),
    
    'valid':
    datasets.ImageFolder(os.path.join(disk_folder, 'valid'), data_transforms['valid'])
}

dataloaders = {
    'train':
    DataLoader(image_datasets['train'],
              batch_size=16,
              shuffle=True,
              num_workers=0),
    
    'valid':
    DataLoader(image_datasets['valid'],
              batch_size=16,
              shuffle=True,
              num_workers=0)
}

"Data Preprocessing COMPLETED"

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")

model = models.resnet18(pretrained=False).to(device)

model.fc = nn.Linear(model.fc.in_features, 33).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters())

In [ ]:
def train_model(model, criterion, optimizer, num_epochs=3):
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase])
            epoch_acc = running_corrects.double() / len(dataloaders[phase])

            print('{} loss: {:.4f}, acc: {:.4f}'.format(phase,
                                                        epoch_loss,
                                                        epoch_acc))
    return model

In [ ]:
trained_model = train_model(model, criterion, optimizer, num_epochs=5)